In [ ]:
RUN_EVALUATION = True # Re-compute results (this might take a long time)
SAVE_FIGURES = True
FIGURE_PATH = "./figures/"
EVENT_LOGS_PATH = "../../"

import copy
import itertools
from typing import Tuple, TypedDict, Literal
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
from pm4py.objects.petri_net.utils.petri_utils import (
    remove_transition,
    get_transition_by_name,
)
from pm4py.objects.petri_net.obj import Marking, PetriNet
import pm4py
# For Alignment MP
import sys

sys.setrecursionlimit(10000)
MP_ALIGNMENTS = False


class RemovingActsResult(TypedDict):
    removedActs: frozenset[str]
    fitness: float
    precision: float


def get_sorted_disconnected_acts(net: PetriNet, log: pd.DataFrame):
    disconnected_acts = {
        t.label
        for t in net.transitions
        if len(t.in_arcs) == 0 and len(t.out_arcs) == 0 and t.label is not None
    }
    acts_with_event_count = pm4py.stats.get_event_attribute_values(log, "concept:name")
    sorted_disconnected_acts = [
        k
        for k, v in sorted(acts_with_event_count.items(), key=lambda item: item[1])
        if k in disconnected_acts
    ]
    return (sorted_disconnected_acts, acts_with_event_count)


# Powerset
def powerset(l: list):
    return itertools.chain.from_iterable(
        itertools.combinations(l, r) for r in range(0, len(l) + 1)
    )


def remove_transitions(
    mode: Literal["greedy", "brute-force"],
    pn: Tuple[PetriNet, Marking, Marking],
    log: pd.DataFrame,
    max_acts_remove = 5
):
    (initial_net, init_im, init_fm) = copy.deepcopy(pn)
    if init_im is None:
        init_im = Marking()
    if init_fm is None:
        init_fm = Marking()

    sorted_disconnected_acts, acts_with_count = get_sorted_disconnected_acts(
        initial_net, log
    )
    # Only consider at most the max_acts_remove most infrequent acts
    sorted_disconnected_acts = sorted_disconnected_acts[0:max_acts_remove+1]

    if mode == "greedy":
        # List of the top i most infrequent activities, for i=0...#disconected_acts
        transition_sets_to_remove = [
            frozenset(sorted_disconnected_acts[0:i])
            for i in range(len(sorted_disconnected_acts) + 1)
        ]   
    else:
        # Powerset (i.e., try out all combinations)
        transition_sets_to_remove = [
            frozenset(s) for s in powerset(sorted_disconnected_acts)
        ]
    print(f"#Iterations: {len(transition_sets_to_remove)}")
    removing_disconnected_acts_data: dict[frozenset[str], RemovingActsResult] = dict()
    for trans_to_rm in transition_sets_to_remove:
        (net, im, fm) = copy.deepcopy((initial_net, init_im, init_fm))
        print(im, fm)
        if trans_to_rm not in removing_disconnected_acts_data:
            for a in trans_to_rm:
                # Assumes that label = name for transition, but for our case this should be fine.
                t = get_transition_by_name(net, a)
                net = remove_transition(net, t)
            fit = pm4py.fitness_alignments(log, net, im, fm, MP_ALIGNMENTS)[
                "average_trace_fitness"
            ]
            prec = pm4py.precision_alignments(log, net, im, fm, MP_ALIGNMENTS)
            print(f"Fitness: {fit}, Precision: {prec}")
            removing_disconnected_acts_data[trans_to_rm] = {
                "removedActs": trans_to_rm,
                "fitness": fit,
                "precision": prec,
            }
        # Do not re-do already evaluated configurations
        else:
            pass
    return removing_disconnected_acts_data


In [ ]:
import json


def dump_json_results(
    name: str, removed_acts_data: dict[frozenset[str], RemovingActsResult]
):
    as_list = [
        {
            **removed_acts_data[k],
            "removedActs": list(removed_acts_data[k]["removedActs"]),
        }
        for k in removed_acts_data
    ]
    with open(FIGURE_PATH + name, "wt") as f:
        json.dump(as_list, f)


In [ ]:
log_to_path = {
    "BPI_Challenge_2020_DomesticDeclarations": f"{EVENT_LOGS_PATH}DomesticDeclarations.xes.gz",
    "BPI_Challenge_2020": f"{EVENT_LOGS_PATH}BPI_Challenge_2020_request_for_payments.xes",
    "BPI_Challenge_2019_sampled_3000cases": f"{EVENT_LOGS_PATH}BPI_Challenge_2019_sampled3000.xes",
    "Sepsis": f"{EVENT_LOGS_PATH}Sepsis Cases - Event Log.xes.gz",
    "RTFM": f"{EVENT_LOGS_PATH}Road_Traffic_Fine_Management_Process.xes.gz",
}
log_to_IMf_res = {
    "BPI_Challenge_2020": "algo_res2023-04-02T21:05:14.839491_BPI_Challenge_2020.pickle",
    "BPI_Challenge_2020_DomesticDeclarations": "algo_res2023-04-02T21:02:52.613449_BPI_Challenge_2020_DomesticDeclarations.pickle",
    "RTFM": "algo_res2023-04-02T23:26:44.105591_RTFM.pickle",
    "Sepsis": "algo_res2023-04-02T23:11:00.988178_Sepsis.pickle",
    "BPI_Challenge_2019_sampled_3000cases": "algo_res2023-04-02T22:08:45.153238_BPI_Challenge_2019_sampled_3000cases.pickle",
}


In [ ]:
loaded_logs = dict()

In [ ]:
import glob
import warnings
ALPHAPPP_PNML_FOLDER = "../../report/report_figures/"
pn_path_per_algolog = dict()
for pnml_path in glob.glob(ALPHAPPP_PNML_FOLDER + "*.pnml"):
    pnml_name = pnml_path.split("/")[-1]
    split_name = pnml_name.split("_")
    split_name.pop() # Pop timestamp
    algo_name = split_name.pop() # Pop algo name
    split_name.pop() # Pop _model
    log_name = "_".join(split_name)
    pn_path_per_algolog[(log_name,algo_name)] = pnml_path
    log_path = log_to_path[log_name]
    pn = pm4py.read_pnml(pnml_path)
    if RUN_EVALUATION:
        n = len(pn[0].transitions)

        disconnected_acts = {t.label for t in pn[0].transitions if len(t.in_arcs) == 0 and len(t.out_arcs) == 0 and t.label is not None}
        k = len(disconnected_acts)
        print(f"{k} out of {n} transitions are disconnected & labeled")
        # print(f"This means that there are {pow(2,k)} subsets of disconnected & labeled transitions to test in brute-force mode.")
        # print("---")
        # if pow(2,k) > 100:
        #     print(f"Skip {pnml_name} because evaluating all subsets is infeasible (#: {pow(2,k)})")
        # elif k <= 1:
        #     print(f"Skip {pnml_name} because there is only 0/1 disconnected labeled transition")
        # else:
        if k > 1:
            print(f"Evaluating for {pnml_name}")
            # Load log (if necessary)
            if log_name in loaded_logs:
                log = loaded_logs[log_name]
            else:
                log_path = log_to_path[log_name]
                with warnings.catch_warnings():
                    # Suppress flood of dt-parsing warnings:
                    # (Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.)
                    warnings.simplefilter(action='ignore', category=UserWarning)
                    log = pm4py.read_xes(log_path)
                loaded_logs[log_name] = log
                print("Finished importing EventLog")
                
            greedy_res = remove_transitions("greedy",pn,log)
            dump_json_results(f"removed-acts-results-{log_name}-{algo_name}-greedy.json", greedy_res)
            print("Greedy mode finished")
            brute_force_res = remove_transitions("brute-force",pn,log)
            dump_json_results(f"removed-acts-results-{log_name}-{algo_name}-brute-force.json", brute_force_res)
            print("Brute-force mode finished")


In [ ]:
  
import pickle
from typing import Optional
import matplotlib
MAIN_PALETTE = ["#1f77b4","#f9bc02","#2ca02c"]
ALT_PALETTE = ["#41cde0","#fcec3f","#5dfc5d"]
MEASURES = ["Fitness","F1-Score","Precision"]
def plot_greedy_act_removal(fig_title: str,removing_disconnected_acts_data: list[RemovingActsResult], compare_to: Optional[list[RemovingActsResult]] = None):
    plt.rc('legend',fontsize=14)
    values = []
    comparison_values = []
    for i in range(len(removing_disconnected_acts_data)):
        # Optionally: Use % of events not covered anymore (bc. activity was removed)
        # Note, that this only makes sense when not using compare_to
        # Additionally requires log frequency info (i.e., disconnected_acts_count object)
        # removed_events = 0
        # for a in datum['removedActs']:
        #     removed_events += disconnected_acts_count[a]
        # removed_events_rel = round(removed_events/len(log)*100,2)
        removed_events_rel = i
        datum = removing_disconnected_acts_data[i]
        fit = datum['fitness']
        prec = datum['precision']
        f1 = 2*(prec * fit)/(prec + fit)
        values.append({'value': fit, "type": "fitness", "x": removed_events_rel})
        values.append({'value': f1, "type": "f1-score", "x": removed_events_rel})
        values.append({'value': prec, "type": "precision", "x": removed_events_rel})
        if compare_to is not None:
            removed_events_rel = i #round(removed_events/len(log)*100,2)
            datum = compare_to[i]
            fit = datum['fitness']
            prec = datum['precision']
            f1 = 2*(prec * fit)/(prec + fit)
            comparison_values.append({'value': fit, "type": "fitness", "x": removed_events_rel})
            comparison_values.append({'value': f1, "type": "f1-score", "x": removed_events_rel})
            comparison_values.append({'value': prec, "type": "precision", "x": removed_events_rel})
    # Add empty values to add space before IMf baseline values
    # We add a "//" patch later, to indicate the axis break
    values.append({'value': 0.0, "type": "fitness", "x": ""})
    values.append({'value': 0.0, "type": "f1-score", "x": ""})
    values.append({'value': 0.0, "type": "precision", "x": ""})
    comparison_values.append({'value': 0.0, "type": "fitness", "x": ""})
    comparison_values.append({'value': 0.0, "type": "f1-score", "x": ""})
    comparison_values.append({'value': 0.0, "type": "precision", "x": ""})
    # Add IMf as baseline comparison
    imf_pickle_path = f"/home/aarkue/doc/sciebo/alpha-revisit/final_figures/{log_to_IMf_res[log_name]}"
    with open(imf_pickle_path,'rb') as f:
        p = pickle.load(f)
        imf_data = p["IMf 0.2"]
        imf_fit = imf_data['fitness (alignment) %']/100
        imf_prec = imf_data['precision (alignment) %']/100
        imf_f1 = 2*(imf_prec * imf_fit)/(imf_prec + imf_fit)
        print(f"IMf 0.2 F1-Score: {round(imf_f1,4)}")
        values.append({'value': imf_fit, "type": "fitness", "x": "IMf 0.2"})
        values.append({'value': imf_f1, "type": "f1-score", "x": "IMf 0.2"})
        values.append({'value': imf_prec, "type": "precision", "x": "IMf 0.2"}) 
        comparison_values.append({'value': 0.0, "type": "fitness", "x": "IMf 0.2"})
        comparison_values.append({'value': 0.0, "type": "f1-score", "x": "IMf 0.2"})
        comparison_values.append({'value': 0.0, "type": "precision", "x": "IMf 0.2"})

    df = pd.DataFrame(values, columns=["value","type","x"])
    fig,ax = plt.subplots(figsize=(15,6))

    bar = sns.barplot(df,y="value",x="x", hue="type",ax=ax, palette=MAIN_PALETTE, linestyle = "-", linewidth = 1, edgecolor = "black")

    if compare_to is not None:
        df_comparison = pd.DataFrame(comparison_values, columns=["value","type","x"])
        bar_comparison = sns.barplot(df_comparison,y="value",x="x", hue="type",ax=ax, palette=ALT_PALETTE, alpha=0.4, fill=True)
        bar_comparison = sns.barplot(df_comparison,y="value",x="x", hue="type",ax=ax, alpha=1.0, linestyle = ":", linewidth = 1, edgecolor = "black", fill=False) 
        plt.suptitle("Greedy and Brute-Force Activity Removal",fontsize=24, y=1.025)

    ax.set_xlabel("Number of Removed Activities // IMf 0.2 for comparison", fontsize=14)
    ax.set_ylabel("Fitness/Precision/F1-Score")
    ax.set_yticks([v/10 for v in range(0,11,1)])
    ax.set_title(fig_title,fontsize=16, y=1.015)
    t = plt.text(len(removing_disconnected_acts_data), -0.000, r'//', fontsize=18, zorder=100, horizontalalignment='center', verticalalignment='center',color="black", backgroundcolor="white")
    # Build legend handles
    # Starting with main elements
    legend_handles = []
    legend_handles += [matplotlib.patches.Patch(fill=False,linewidth=0,label="")]
    legend_handles += [matplotlib.patches.Patch(fill=False,linewidth=0,label="Greedy Mode")]
    legend_handles += [matplotlib.patches.Patch(color=MAIN_PALETTE[i],label=MEASURES[i]) for i in range(3)]
    # + Comparison legend handles
    if compare_to is not None:
        legend_handles += [matplotlib.patches.Patch(fill=False,linewidth=0,label="")]
        legend_handles += [matplotlib.patches.Patch(fill=False,linewidth=0,label="Brute-Force Mode\n  (Best F1-Score)")]
        legend_handles += [matplotlib.patches.Patch(facecolor=[c+"35" for c in ALT_PALETTE][i],linestyle=":", edgecolor = "black", linewidth=1,label=[m + " (brute-force)" for m in MEASURES][i]) for i in range(3)]
    # Make + move legend
    legend = plt.legend(handles=legend_handles,loc=1, title="Scores and Approaches",title_fontsize=16)
    sns.move_legend(ax,loc=(1.01, 0.33 ))
    # Add explanation text
    plt.text(min(0.4,0.1 * len(removing_disconnected_acts_data)/4),-0.25,"""Greedy mode: Remove disconnected labeled transitions in the reverse order of their log frequency, one by one.
Brute-force mode: Consider all possible subsets of disconnected labeled transitions.
For brute-force mode, the activity subset (of set size) which removal leads to the best F1-Score is considered.""", fontsize=12)
    return fig


In [ ]:
import glob
import json
# Helper to quickly generate comparison figures for all available results (dumped as json)
for path in glob.glob(f"{FIGURE_PATH}/removed-acts-results-*-brute-force.json"):
    # Retrieve log and algo name from filename
    path_prefix = path[0:-len("-brute-force.json")]
    name: str = path_prefix.split("/")[-1][len("removed-acts-results-"):]
    log_name = name[0:name.index("α")-1]
    algo_name = name[name.index("α"):]
    print(name)
    brute_force_path = path
    greedy_path = path_prefix + "-greedy.json"
    with open(greedy_path, "rt") as f:
        greedy_data: list[RemovingActsResult] = json.load(f)
    with open(brute_force_path, "rt") as f:
        brute_force_data: list[RemovingActsResult] = json.load(f)

    best_brute_force = [
        sorted([x for x in brute_force_data if len(x['removedActs']) == i],
               key=lambda e: 2*(e['precision'] * e['fitness'])/(e['precision'] + e['fitness']), reverse=True)
        [0] for i in range(len(greedy_data))]

    fig_title = f"Log: {log_name}, Algorithm: {algo_name}"
    fig: plt.Figure = plot_greedy_act_removal(
        fig_title, greedy_data, best_brute_force)
    if SAVE_FIGURES:
        fig.savefig(FIGURE_PATH + name + ".svg", bbox_inches='tight')
    fig.show()

    print("---")